# Deep Agents: Building Complex Agents for Long-Horizon Tasks

In this notebook, we'll explore **Deep Agents** - a new approach to building AI agents that can handle complex, multi-step tasks over extended periods. We'll implement all four key elements of Deep Agents while building on our Personal Wellness Assistant use case.

**Learning Objectives:**
- Understand the four key elements of Deep Agents: Planning, Context Management, Subagent Spawning, and Long-term Memory
- Implement each element progressively using the `deepagents` package
- Learn to use Skills for progressive capability disclosure
- Use the `deepagents-cli` for interactive agent sessions

## Table of Contents:

- **Breakout Room #1:** Deep Agent Foundations
  - Task 1: Dependencies & Setup
  - Task 2: Understanding Deep Agents
  - Task 3: Planning with Todo Lists
  - Task 4: Context Management with File Systems
  - Task 5: Basic Deep Agent
  - Question #1 & Question #2
  - Activity #1: Build a Research Agent

- **Breakout Room #2:** Advanced Features & Integration
  - Task 6: Subagent Spawning
  - Task 7: Long-term Memory Integration
  - Task 8: Skills - On-Demand Capabilities
  - Task 9: Using deepagents-cli
  - Task 10: Building a Complete Deep Agent System
  - Question #3 & Question #4
  - Activity #2: Build a Wellness Coach Agent

---
# Breakout Room #1
## Deep Agent Foundations

## Task 1: Dependencies & Setup

Before we begin, make sure you have:

1. **API Keys** for:
   - Anthropic (default for Deep Agents) or OpenAI
   - LangSmith (optional, for tracing)
   - Tavily (optional, for web search)

2. **Dependencies installed** via `uv sync`

3. **For the CLI** (Task 9): `uv pip install deepagents-cli`

### Environment Setup

You can either:
- Create a `.env` file with your API keys (recommended):
  ```
  ANTHROPIC_API_KEY=your_key_here
  OPENAI_API_KEY=your_key_here
  LANGCHAIN_API_KEY=your_key_here
  ```
- Or enter them interactively when prompted

In [3]:
# Core imports
import os
import getpass
from uuid import uuid4
from typing import Annotated, TypedDict, Literal

import nest_asyncio
nest_asyncio.apply()  # Required for async operations in Jupyter

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

def get_api_key(env_var: str, prompt: str) -> str:
    """Get API key from environment or prompt user."""
    value = os.environ.get(env_var, "")
    if not value:
        value = getpass.getpass(prompt)
        if value:
            os.environ[env_var] = value
    return value

In [4]:
# Set Anthropic API Key (default for Deep Agents)
anthropic_key = get_api_key("ANTHROPIC_API_KEY", "Anthropic API Key: ")
if anthropic_key:
    print("Anthropic API key set")
else:
    print("Warning: No Anthropic API key configured")

Anthropic API key set


In [5]:
# Optional: OpenAI for alternative models and subagents
openai_key = get_api_key("OPENAI_API_KEY", "OpenAI API Key (press Enter to skip): ")
if openai_key:
    print("OpenAI API key set")
else:
    print("OpenAI API key not configured (optional)")

OpenAI API key set


In [6]:
# Optional: LangSmith for tracing
langsmith_key = get_api_key("LANGCHAIN_API_KEY", "LangSmith API Key (press Enter to skip): ")

if langsmith_key:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = f"AIE9 - Deep Agents - {uuid4().hex[0:8]}"
    print(f"LangSmith tracing enabled. Project: {os.environ['LANGCHAIN_PROJECT']}")
else:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    print("LangSmith tracing disabled")

LangSmith tracing enabled. Project: AIE9 - Deep Agents - b807f6a8


In [7]:
# Verify deepagents installation
from deepagents import create_deep_agent
print("deepagents package imported successfully!")

# Test with a simple agent
test_agent = create_deep_agent()
result = test_agent.invoke({
    "messages": [{"role": "user", "content": "Say 'Deep Agents ready!' in exactly those words."}]
})
print(result["messages"][-1].content)

deepagents package imported successfully!
Deep Agents ready!


## Task 2: Understanding Deep Agents

**Deep Agents** represent a shift from simple tool-calling loops to sophisticated agents that can handle complex, long-horizon tasks. They address four key challenges:

### The Four Key Elements

| Element | Challenge Addressed | Implementation |
|---------|---------------------|----------------|
| **Planning** | "What should I do?" | Todo lists that persist task state |
| **Context Management** | "What do I know?" | File systems for storing/retrieving info |
| **Subagent Spawning** | "Who can help?" | Task tool for delegating to specialists |
| **Long-term Memory** | "What did I learn?" | LangGraph Store for cross-session memory |

### Deep Agents vs Traditional Agents

```
Traditional Agent Loop:
┌─────────────────────────────────────┐
│  User Query                         │
│       ↓                             │
│  Think → Act → Observe → Repeat     │
│       ↓                             │
│  Response                           │
└─────────────────────────────────────┘
Problems: Context bloat, no delegation,
          loses track of complex tasks

Deep Agent Architecture:
┌─────────────────────────────────────────────────────────┐
│                    Deep Agent                           │
├─────────────────────────────────────────────────────────┤
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐   │
│  │   PLANNING   │  │   CONTEXT    │  │   MEMORY     │   │
│  │              │  │  MANAGEMENT  │  │              │   │
│  │ write_todos  │  │              │  │   Store      │   │
│  │ update_todo  │  │  read_file   │  │  namespace   │   │
│  │ list_todos   │  │  write_file  │  │  get/put     │   │
│  │              │  │  edit_file   │  │              │   │
│  └──────────────┘  │  ls          │  └──────────────┘   │
│                    └──────────────┘                     │
│  ┌──────────────────────────────────────────────────┐   │
│  │              SUBAGENT SPAWNING                   │   │
│  │                                                  │   │
│  │  task(prompt, tools, model, system_prompt)       │   │
│  │       ↓              ↓              ↓            │   │
│  │  ┌────────┐    ┌────────┐    ┌────────┐          │   │
│  │  │Research│    │Writing │    │Analysis│          │   │
│  │  │Subagent│    │Subagent│    │Subagent│          │   │
│  │  └────────┘    └────────┘    └────────┘          │   │
│  └──────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────┘
```

### When to Use Deep Agents

| Use Case | Traditional Agent | Deep Agent |
|----------|-------------------|------------|
| Simple Q&A | ✅ | Overkill |
| Single-step tool use | ✅ | Overkill |
| Multi-step research | ⚠️ May lose track | ✅ |
| Complex projects | ❌ Context overflow | ✅ |
| Parallel task execution | ❌ | ✅ |
| Long-running sessions | ❌ | ✅ |

### Key Insight: "Planning is Context Engineering"

Deep Agents treat planning not as a separate phase, but as **context engineering**:
- Todo lists aren't just task trackers—they're **persistent context** about what to do
- File systems aren't just storage—they're **extended memory** beyond the context window
- Subagents aren't just helpers—they're **context isolation** to prevent bloat

## Task 3: Planning with Todo Lists

The first key element of Deep Agents is **Planning**. Instead of trying to hold all task state in the conversation, Deep Agents use structured todo lists.

### Why Todo Lists?

1. **Persistence**: Tasks survive across conversation turns
2. **Visibility**: Both agent and user can see progress
3. **Structure**: Clear tracking of what's done vs pending
4. **Recovery**: Agent can resume from where it left off

### Todo List Tools

| Tool | Purpose |
|------|----------|
| `write_todos` | Create a structured task list |
| `update_todo` | Mark tasks as complete/in-progress |
| `list_todos` | View current task state |

In [8]:
from langchain_core.tools import tool
from typing import List, Optional
import json

# Simple in-memory todo storage for demonstration
# In production, Deep Agents use persistent storage
TODO_STORE = {}

@tool
def write_todos(todos: List[dict]) -> str:
    """Create a list of todos for tracking task progress.
    
    Args:
        todos: List of todo items, each with 'title' and optional 'description'
    
    Returns:
        Confirmation message with todo IDs
    """
    created = []
    for i, todo in enumerate(todos):
        todo_id = f"todo_{len(TODO_STORE) + i + 1}"
        TODO_STORE[todo_id] = {
            "id": todo_id,
            "title": todo.get("title", "Untitled"),
            "description": todo.get("description", ""),
            "status": "pending"
        }
        created.append(todo_id)
    return f"Created {len(created)} todos: {', '.join(created)}"

@tool
def update_todo(todo_id: str, status: Literal["pending", "in_progress", "completed"]) -> str:
    """Update the status of a todo item.
    
    Args:
        todo_id: The ID of the todo to update
        status: New status (pending, in_progress, completed)
    
    Returns:
        Confirmation message
    """
    if todo_id not in TODO_STORE:
        return f"Todo {todo_id} not found"
    TODO_STORE[todo_id]["status"] = status
    return f"Updated {todo_id} to {status}"

@tool
def list_todos() -> str:
    """List all todos with their current status.
    
    Returns:
        Formatted list of all todos
    """
    if not TODO_STORE:
        return "No todos found"
    
    result = []
    for todo_id, todo in TODO_STORE.items():
        status_emoji = {"pending": "⬜", "in_progress": "🔄", "completed": "✅"}
        emoji = status_emoji.get(todo["status"], "❓")
        result.append(f"{emoji} [{todo_id}] {todo['title']} ({todo['status']})")
    return "\n".join(result)

print("Todo tools defined!")

Todo tools defined!


In [10]:
# Test the todo tools
TODO_STORE.clear()  # Reset for demo

# Create some wellness todos
result = write_todos.invoke({
    "todos": [
        {"title": "Assess current sleep patterns", "description": "Review user's sleep schedule and quality"},
        {"title": "Research sleep improvement strategies", "description": "Find evidence-based techniques"},
        {"title": "Create personalized sleep plan", "description": "Combine findings into actionable steps"},
    ]
})
print(result)
print("\nCurrent todos:")
print(list_todos.invoke({}))

Created 3 todos: todo_1, todo_3, todo_5

Current todos:
⬜ [todo_1] Assess current sleep patterns (pending)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


In [11]:
# Simulate progress
update_todo.invoke({"todo_id": "todo_1", "status": "completed"})
update_todo.invoke({"todo_id": "todo_2", "status": "in_progress"})

print("After updates:")
print(list_todos.invoke({}))

After updates:
✅ [todo_1] Assess current sleep patterns (completed)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


## Task 4: Context Management with File Systems

The second key element is **Context Management**. Deep Agents use file systems to:

1. **Offload large content** - Store research, documents, and results to disk
2. **Persist across sessions** - Files survive beyond conversation context
3. **Share between subagents** - Subagents can read/write shared files
4. **Prevent context overflow** - Large tool results automatically saved to disk

### Automatic Context Management

Deep Agents automatically handle context limits:
- **Large result offloading**: Tool results >20k tokens → saved to disk
- **Proactive offloading**: At 85% context capacity → agent saves state to disk
- **Summarization**: Long conversations get summarized while preserving intent

### File System Tools

| Tool | Purpose |
|------|----------|
| `ls` | List directory contents |
| `read_file` | Read file contents |
| `write_file` | Create/overwrite files |
| `edit_file` | Make targeted edits |

In [14]:
import os
from pathlib import Path

# Create a workspace directory for our agent
WORKSPACE = Path("workspace")
WORKSPACE.mkdir(exist_ok=True)

@tool
def ls(path: str = ".") -> str:
    """List contents of a directory.
    
    Args:
        path: Directory path to list (default: current directory)
    
    Returns:
        List of files and directories
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"Directory not found: {path}"
    
    items = []
    for item in sorted(target.iterdir()):
        prefix = "[DIR]" if item.is_dir() else "[FILE]"
        size = f" ({item.stat().st_size} bytes)" if item.is_file() else ""
        items.append(f"{prefix} {item.name}{size}")
    
    return "\n".join(items) if items else "(empty directory)"

@tool
def read_file(path: str) -> str:
    """Read contents of a file.
    
    Args:
        path: Path to the file to read
    
    Returns:
        File contents
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    return target.read_text()

@tool
def write_file(path: str, content: str) -> str:
    """Write content to a file (creates or overwrites).
    
    Args:
        path: Path to the file to write
        content: Content to write to the file
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    target.parent.mkdir(parents=True, exist_ok=True)
    target.write_text(content)
    return f"Wrote {len(content)} characters to {path}"

@tool
def edit_file(path: str, old_text: str, new_text: str) -> str:
    """Edit a file by replacing text.
    
    Args:
        path: Path to the file to edit
        old_text: Text to find and replace
        new_text: Replacement text
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    
    content = target.read_text()
    if old_text not in content:
        return f"Text not found in {path}"
    
    new_content = content.replace(old_text, new_text, 1)
    target.write_text(new_content)
    return f"Updated {path}"

print("File system tools defined!")
print(f"Workspace: {WORKSPACE.absolute()}")

File system tools defined!
Workspace: /Users/sireeshapulipati/AIE9/07_Deep_Agents/workspace


In [15]:
# Test the file system tools
print("Current workspace contents:")
print(ls.invoke({"path": "."}))

Current workspace contents:
[FILE] Alex_2Week_Integrated_Wellness_Program.txt (7770 bytes)
[FILE] alexs_2_week_vegetarian_meal_plan.txt (5245 bytes)
[DIR] data
[FILE] exercise_program_alex.txt (4087 bytes)
[DIR] exercise_programs
[FILE] morning_routine_guide.md (45226 bytes)
[FILE] personalized_sleep_improvement_plan.md (5510 bytes)
[DIR] research
[FILE] stress_management_sleep_optimization_plan_for_Alex.txt (3478 bytes)
[FILE] stress_management_sleep_optimization_program.txt (6961 bytes)
[DIR] workspace


In [16]:
# Create a research notes file
notes = """# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations
"""

result = write_file.invoke({"path": "research/sleep_notes.md", "content": notes})
print(result)

# Verify it was created
print("\nResearch directory:")
print(ls.invoke({"path": "research"}))

Wrote 242 characters to research/sleep_notes.md

Research directory:
[FILE] comprehensive_sleep_improvement_guide.md (14237 bytes)
[FILE] gradual_implementation_strategies.md (12212 bytes)
[FILE] morning_energy_fatigue_reduction.md (8404 bytes)
[FILE] sleep_hygiene_best_practices.md (10457 bytes)
[FILE] sleep_notes.md (242 bytes)
[FILE] sleep_schedule_optimization.md (3755 bytes)
[FILE] technology_screen_management.md (5551 bytes)


In [17]:
# Read and edit the file
print("File contents:")
print(read_file.invoke({"path": "research/sleep_notes.md"}))

File contents:
# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations



## Task 5: Basic Deep Agent

Now let's create a basic Deep Agent using the `deepagents` package. This combines:
- Planning (todo lists)
- Context management (file system)
- A capable LLM backbone

### Configuring the FilesystemBackend

Deep Agents come with **built-in file tools** (`ls`, `read_file`, `write_file`, `edit_file`). To control where files are stored, we configure a `FilesystemBackend`:

```python
from deepagents.backends import FilesystemBackend

backend = FilesystemBackend(
    root_dir="/path/to/workspace",
    virtual_mode=True  # REQUIRED to actually sandbox files!
)
```

**Critical: `virtual_mode=True`**
- Without `virtual_mode=True`, agents can still write anywhere on the filesystem!
- The `root_dir` alone does NOT restrict file access
- `virtual_mode=True` blocks paths with `..`, `~`, and absolute paths outside root

In [18]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Configure the filesystem backend to use our workspace directory
# IMPORTANT: virtual_mode=True is required to actually restrict paths to root_dir
# Without it, agents can still write anywhere on the filesystem!
workspace_path = Path("workspace").absolute()
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True  # This is required to sandbox file operations!
)

# Combine our custom tools (for todo tracking)
# Note: Deep Agents has built-in file tools (ls, read_file, write_file, edit_file)
# that will use the configured FilesystemBackend
custom_tools = [
    write_todos,
    update_todo,
    list_todos,
]

# Create a basic Deep Agent
wellness_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=custom_tools,
    backend=filesystem_backend,  # Configure where files are stored
    system_prompt="""You are a Personal Wellness Assistant that helps users improve their health.

When given a complex task:
1. First, create a todo list to track your progress
2. Work through each task, updating status as you go
3. Save important findings to files for reference
4. Provide a clear summary when complete

Be thorough but concise. Always explain your reasoning."""
)

print(f"Basic Deep Agent created!")
print(f"File operations sandboxed to: {workspace_path}")

Basic Deep Agent created!
File operations sandboxed to: /Users/sireeshapulipati/AIE9/07_Deep_Agents/workspace


In [16]:
# Reset todo store for fresh demo
TODO_STORE.clear()

# Test with a multi-step wellness task
result = wellness_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please create a personalized sleep improvement plan for me and save it to a file."""
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Perfect! I've created a comprehensive, personalized sleep improvement plan that directly addresses your specific challenges. Here's what I've delivered:

## Your Personalized Sleep Plan Summary:

**📁 File Saved**: `/personalized_sleep_improvement_plan.md`

### Key Features of Your Plan:

1. **Gradual 4-Phase Approach** (16 weeks total):
   - **Phase 1**: Foundation building and environment optimization
   - **Phase 2**: Core habit installation with gradual bedtime adjustment
   - **Phase 3**: Morning energy enhancement
   - **Phase 4**: System optimization and resilience building

2. **Addresses Your Specific Issues**:
   - **Inconsistent bedtime**: Gradual 15-20 minute adjustments targeting 10:30pm
   - **Phone in bed**: Complete digital boundary with charging station outside bedroom
   - **Morning fatigue**: Enhanced morning routine with light exposure and hydration

3. **Practical Implementation**:
   - Start with just 2-3 changes to avoid overwhelm
   - 80% consiste

In [17]:
# Check what the agent created
print("Todo list after task:")
print(list_todos.invoke({}))

print("\n" + "="*50)
print("\nWorkspace contents:")
# List files in the workspace directory
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    else:
        print(f"  [DIR] {f.name}/")

Todo list after task:
✅ [todo_1] Analyze current sleep issues (completed)
✅ [todo_3] Research evidence-based sleep improvement strategies (completed)
✅ [todo_5] Research sleep schedule optimization strategies (completed)
✅ [todo_7] Save plan to file (completed)
✅ [todo_6] Research technology and screen time management (completed)
✅ [todo_8] Research morning energy and fatigue reduction strategies (completed)
✅ [todo_10] Research sleep hygiene best practices (completed)
✅ [todo_12] Research gradual implementation and sustainability strategies (completed)
✅ [todo_14] Organize and structure findings by category (completed)


Workspace contents:
  [FILE] personalized_sleep_improvement_plan.md (5510 bytes)
  [DIR] research/


---
## ❓ Question #1:

What are the **trade-offs** of using todo lists for planning? Consider:
- When might explicit planning overhead slow things down?
- How granular should todo items be?
- What happens if the agent creates todos but never completes them?

##### Answer:
Explicit planning is useful only for complex long-running tasks. Because planning adds overhead, and slows down the final response. 
If a task is simple, explicit to-do lists result in overcomplicating and increase cost without any benefit.

Todo item should be specific, actionable, and corse enough to justify the overhead of breaking down (i.e., not too fine-grained)

If the agent never completes todos, it means the agent might be stuck on an infinite loop, or has run into any issue. It results in resource exhaustion or too much resource usage. 

## ❓ Question #2:

How would you design a **context management strategy** for a wellness agent that:
- Needs to reference a large health document (16KB)
- Tracks user metrics over time
- Must remember user conditions (allergies, medications) for safety

What goes in files vs. in the prompt? What should never be offloaded?

##### Answer:
- can use sematic retrieval with the health document to retrieve only relevant information for the task at hand
- for tracking user metrics over time, the agent should leevrage semantic memory per user
- user conditions also need to be stored in semantic memory and referenced in system prompt

---
## 🏗️ Activity #1: Build a Research Agent

Build a Deep Agent that can research a wellness topic and produce a structured report.

### Requirements:
1. Create todos for the research process
2. Read from the HealthWellnessGuide.txt in the data folder
3. Save findings to a structured markdown file
4. Update todo status as tasks complete

### Test prompt:
"Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."

In [20]:
### YOUR CODE HERE ###

# Step 1: Create a research agent with appropriate tools
# Hint: You'll need file tools to read the wellness guide
research_tools = [write_todos, update_todo, list_todos]

# Step 2: Add a tool to read from the data folder
# Hint: Use Path("data/HealthWellnessGuide.txt")
@tool
def read_health_wellness_guide():
    """Read the HealthWellnessGuide.txt file"""
    return read_file.invoke({"path": "data/HealthWellnessGuide.txt"})

# Step 3: Create the agent with a research-focused system prompt
wellness_research_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=research_tools + [read_health_wellness_guide],
    backend=filesystem_backend,
    system_prompt="""You are a wellness research specialist. Your job is to research wellness topics and produce structured, evidence-based reports.

Process:
1. For any research request, first create a todo list with write_todos (e.g. read the wellness guide, extract relevant information, synthesize findings, write the report, save to file).
2. Work through each task in order. Use update_todo to mark tasks in_progress and completed as you go. Use list_todos when you need to check progress.
3. Read the wellness guide using the read_health_wellness_guide tool (or read_file with path "data/HealthWellnessGuide.txt") to get evidence-based information.
4. Save your final report to a markdown file in the workspace (e.g. workspace/research/<topic>_guide.md) using write_file. Include at least 5 strategies or sections when the user asks for a comprehensive guide.
5. Cite the source (HealthWellnessGuide) when possible. Be thorough but concise."""
)

# Step 4: Test with the stress management research task
TODO_STORE.clear()

result = wellness_research_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """ 
        Research stress management techniques 
        and create a comprehensive guide with at least 5 evidence-based strategies.
        """
    }]
})
print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Perfect! I have successfully completed your comprehensive stress management guide. Here's what I've accomplished:

## ✅ Research Complete: Comprehensive Stress Management Guide

I've created a thorough, evidence-based stress management guide with **6 comprehensive strategies** (exceeding your requirement of 5), drawing from multiple wellness sources including:

- Established stress management and sleep optimization programs
- Morning routine research and energy optimization techniques
- Sleep hygiene best practices
- Workplace wellness strategies

### The 6 Evidence-Based Strategies Include:

1. **Mindfulness and Breathing Techniques** - Including the scientifically-proven 4-4-4 breathing method and workplace mini-meditations

2. **Physical Movement and Exercise** - Morning energy activation, stress-relief movement, and workplace mobility strategies

3. **Sleep Optimization and Evening Routines** - Environmental optimization, sleep hygiene practices, and advanced restor

---
# 🤝 Breakout Room #2
## Advanced Features & Integration

## Task 6: Subagent Spawning

The third key element is **Subagent Spawning**. This allows a Deep Agent to delegate tasks to specialized subagents.

### Why Subagents?

1. **Context Isolation**: Each subagent has its own context window, preventing bloat
2. **Specialization**: Different subagents can have different tools/prompts
3. **Parallelism**: Multiple subagents can work simultaneously
4. **Cost Optimization**: Use cheaper models for simpler subtasks

### How Subagents Work

```
Main Agent
    ├── task("Research sleep science", model="gpt-4o-mini")
    │       └── Returns: Summary of findings
    │
    ├── task("Analyze user's sleep data", tools=[analyze_tool])
    │       └── Returns: Analysis results
    │
    └── task("Write recommendations", system_prompt="Be concise")
            └── Returns: Final recommendations
```

Key benefit: The main agent only receives **summaries**, not all the intermediate context!

In [21]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Define specialized subagent configurations
# Note: Subagents inherit the backend from the parent agent
research_subagent = {
    "name": "research-agent",
    "description": "Use this agent to research wellness topics in depth. It can read documents and synthesize information.",
    "system_prompt": """You are a wellness research specialist. Your job is to:
1. Find relevant information in provided documents
2. Synthesize findings into clear summaries
3. Cite sources when possible

Be thorough but concise. Focus on evidence-based information.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",  # Cheaper model for research
}

writing_subagent = {
    "name": "writing-agent",
    "description": "Use this agent to create well-structured documents, plans, and guides.",
    "system_prompt": """You are a wellness content writer. Your job is to:
1. Take research findings and turn them into clear, actionable content
2. Structure information for easy understanding
3. Use formatting (headers, bullets, etc.) effectively

Write in a supportive, encouraging tone.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "anthropic:claude-sonnet-4-20250514",
}

print("Subagent configurations defined!")

Subagent configurations defined!


In [23]:
# Create a coordinator agent that can spawn subagents
coordinator_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[write_todos, update_todo, list_todos],
    backend=filesystem_backend,  # Use the same backend - subagents inherit it
    subagents=[research_subagent, writing_subagent],
    system_prompt="""You are a Wellness Project Coordinator. Your role is to:
1. Break down complex wellness requests into subtasks
2. Delegate research to the research-agent
3. Delegate content creation to the writing-agent
4. Coordinate the overall workflow using todos

Use subagents for specialized work rather than doing everything yourself.
This keeps the work organized and the results high-quality."""
)

print("Coordinator agent created with subagent capabilities!")

Coordinator agent created with subagent capabilities!


In [20]:
# Reset for demo
TODO_STORE.clear()

# Test the coordinator with a complex task
result = coordinator_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Create a comprehensive morning routine guide for better energy.
        
The guide should:
1. Research the science behind morning routines
2. Include practical steps for exercise, nutrition, and mindset
3. Be saved as a well-formatted markdown file"""
    }]
})

print("Coordinator response:")
print(result["messages"][-1].content)

Coordinator response:
Perfect! I've successfully completed your comprehensive morning routine guide for better energy. Here's what I've delivered:

## 🎉 Project Complete! 

Your **Ultimate Morning Routine Guide for Better Energy** has been created and saved as `/morning_routine_guide.md`. 

### ✅ What's Included:

**🔬 Science-Based Foundation:**
- Circadian rhythm research and biological mechanisms
- Cortisol awakening response and adenosine clearance
- Evidence-based benefits from peer-reviewed studies
- Individual differences and personalization strategies

**🏗️ Complete Framework with 3 Core Pillars:**

1. **🏃‍♀️ Exercise Component** - Movement options from 5-minute energy boosters to full 20+ minute sessions
2. **🍎 Nutrition Component** - Strategic hydration protocols, breakfast approaches, and smart caffeine timing
3. **🧠 Mindset Component** - Meditation, journaling, planning, and mental preparation techniques

**📋 Practical Implementation:**
- 4-phase step-by-step implementation 

In [21]:
# Check the results
print("Final todo status:")
print(list_todos.invoke({}))

print("\nGenerated files in workspace:")
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

Final todo status:
✅ [todo_1] Research the science behind morning routines (completed)
✅ [todo_3] Create comprehensive morning routine guide (completed)
✅ [todo_5] Save the final guide as a markdown file (completed)

Generated files in workspace:
  [FILE] morning_routine_guide.md (45226 bytes)
  [FILE] personalized_sleep_improvement_plan.md (5510 bytes)
  [DIR] research/


## Task 7: Long-term Memory Integration

The fourth key element is **Long-term Memory**. Deep Agents integrate with LangGraph's Store for persistent memory across sessions.

### Memory Types in Deep Agents

| Type | Scope | Use Case |
|------|-------|----------|
| **Thread Memory** | Single conversation | Current session context |
| **User Memory** | Across threads, per user | User preferences, history |
| **Shared Memory** | Across all users | Common knowledge, learned patterns |

### Integration with LangGraph Store

Deep Agents can use the same `InMemoryStore` (or `PostgresStore`) we learned in Session 6:

In [24]:
from langgraph.store.memory import InMemoryStore

# Create a memory store
memory_store = InMemoryStore()

# Store user profile
user_id = "user_alex"
profile_namespace = (user_id, "profile")

memory_store.put(profile_namespace, "name", {"value": "Alex"})
memory_store.put(profile_namespace, "goals", {
    "primary": "improve energy levels",
    "secondary": "better sleep"
})
memory_store.put(profile_namespace, "conditions", {
    "dietary": ["vegetarian"],
    "medical": ["mild anxiety"]
})
memory_store.put(profile_namespace, "preferences", {
    "exercise_time": "evening",
    "communication_style": "concise"
})

print(f"Stored profile for {user_id}")

# Retrieve and display
for item in memory_store.search(profile_namespace):
    print(f"  {item.key}: {item.value}")

Stored profile for user_alex
  name: {'value': 'Alex'}
  goals: {'primary': 'improve energy levels', 'secondary': 'better sleep'}
  conditions: {'dietary': ['vegetarian'], 'medical': ['mild anxiety']}
  preferences: {'exercise_time': 'evening', 'communication_style': 'concise'}


In [25]:
# Create memory-aware tools
from langgraph.store.base import BaseStore

@tool
def get_user_profile(user_id: str) -> str:
    """Retrieve a user's wellness profile from long-term memory.
    
    Args:
        user_id: The user's unique identifier
    
    Returns:
        User profile as formatted text
    """
    namespace = (user_id, "profile")
    items = list(memory_store.search(namespace))
    
    if not items:
        return f"No profile found for {user_id}"
    
    result = [f"Profile for {user_id}:"]
    for item in items:
        result.append(f"  {item.key}: {item.value}")
    return "\n".join(result)

@tool
def save_user_preference(user_id: str, key: str, value: str) -> str:
    """Save a user preference to long-term memory.
    
    Args:
        user_id: The user's unique identifier
        key: The preference key
        value: The preference value
    
    Returns:
        Confirmation message
    """
    namespace = (user_id, "preferences")
    memory_store.put(namespace, key, {"value": value})
    return f"Saved preference '{key}' for {user_id}"

print("Memory tools defined!")

Memory tools defined!


In [25]:
# Create a memory-enhanced agent
memory_tools = [
    get_user_profile,
    save_user_preference,
    write_todos,
    update_todo,
    list_todos,
]

memory_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=memory_tools,
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a Personal Wellness Assistant with long-term memory.

At the start of each conversation:
1. Check the user's profile to understand their goals and conditions
2. Personalize all advice based on their profile
3. Save any new preferences they mention

Always reference stored information to show you remember the user."""
)

print("Memory-enhanced agent created!")

Memory-enhanced agent created!


In [26]:
# Test the memory agent
TODO_STORE.clear()

result = memory_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Hi! My user_id is user_alex. What exercise routine would you recommend for me?"
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Hi Alex! Based on your profile, I see you want to improve energy levels and better sleep, prefer evening workouts, and manage mild anxiety. Here's a personalized exercise routine for you:

## Evening Energy & Sleep Routine

**Strength Training (3x/week - Mon/Wed/Fri)**
- 20-30 minutes, 2-3 hours before bed
- Bodyweight exercises: squats, push-ups, lunges, planks
- Light weights if available
- Focus on controlled movements (great for anxiety)

**Active Recovery (2x/week - Tue/Thu)**
- Evening yoga or gentle stretching (15-20 min)
- Walking or light cycling
- Helps transition to rest mode

**Weekend Options**
- Longer nature walks (anxiety-reducing)
- Swimming if accessible
- Dance or movement you enjoy

**Timing Tips:**
- Finish intense workouts 2-3 hours before bed
- End with 5-10 minutes of stretching or breathing exercises
- This timing supports better sleep while boosting daily energy

The routine balances energy-boosting strength work with anxiety-managing gentle mo

## Task 8: Skills - On-Demand Capabilities

**Skills** are a powerful feature for progressive capability disclosure. Instead of loading all tools upfront, agents can load specialized capabilities on demand.

### Why Skills?

1. **Context Efficiency**: Don't waste context on unused tool descriptions
2. **Specialization**: Skills can include detailed instructions for specific tasks
3. **Modularity**: Easy to add/remove capabilities
4. **Discoverability**: Agent can browse available skills

### SKILL.md Format

Skills are defined in markdown files with YAML frontmatter:

```markdown
---
name: skill-name
description: What this skill does
version: 1.0.0
tools:
  - tool1
  - tool2
---

# Skill Instructions

Detailed steps for how to use this skill...
```

In [27]:
# Let's look at the skills we created
skills_dir = Path("skills")

print("Available skills:")
for skill_dir in skills_dir.iterdir():
    if skill_dir.is_dir():
        skill_file = skill_dir / "SKILL.md"
        if skill_file.exists():
            content = skill_file.read_text()
            # Extract name and description from frontmatter
            lines = content.split("\n")
            name = ""
            desc = ""
            for line in lines:
                if line.startswith("name:"):
                    name = line.split(":", 1)[1].strip()
                if line.startswith("description:"):
                    desc = line.split(":", 1)[1].strip()
            print(f"  - {name}: {desc}")

Available skills:
  - meal-planning: Create personalized meal plans based on dietary needs and preferences
  - wellness-assessment: Assess user wellness goals and create personalized recommendations


In [28]:
# Read the wellness-assessment skill
skill_content = Path("skills/wellness-assessment/SKILL.md").read_text()
print(skill_content)

---
name: wellness-assessment
description: Assess user wellness goals and create personalized recommendations
version: 1.0.0
tools:
  - read_file
  - write_file
---

# Wellness Assessment Skill

You are conducting a comprehensive wellness assessment. Follow these steps:

## Step 1: Gather Information
Ask the user about:
- Current health goals (weight, fitness, stress, sleep)
- Any medical conditions or limitations
- Current exercise routine (or lack thereof)
- Dietary preferences and restrictions
- Sleep patterns and quality
- Stress levels and sources

## Step 2: Analyze Responses
Review the user's answers and identify:
- Primary wellness priority
- Secondary goals
- Potential barriers to success
- Existing healthy habits to build on

## Step 3: Create Assessment Report
Write a wellness assessment report to `workspace/wellness_assessment.md` containing:
- Summary of current wellness state
- Identified strengths
- Areas for improvement
- Recommended focus areas (prioritized)
- Suggeste

In [27]:
# Create a skill-aware tool
@tool
def load_skill(skill_name: str) -> str:
    """Load a skill's instructions for a specialized task.
    
    Available skills:
    - wellness-assessment: Assess user wellness and create recommendations
    - meal-planning: Create personalized meal plans
    
    Args:
        skill_name: Name of the skill to load
    
    Returns:
        Skill instructions
    """
    skill_path = Path(f"skills/{skill_name}/SKILL.md")
    if not skill_path.exists():
        available = [d.name for d in Path("skills").iterdir() if d.is_dir()]
        return f"Skill '{skill_name}' not found. Available: {', '.join(available)}"
    
    return skill_path.read_text()

print("Skill loader defined!")

Skill loader defined!


In [30]:
# Create an agent that can load and use skills
skill_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        load_skill,
        write_todos,
        update_todo,
        list_todos,
    ],
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a wellness assistant with access to specialized skills.

When a user asks for something that matches a skill:
1. Load the appropriate skill using load_skill()
2. Follow the skill's instructions carefully
3. Save outputs as specified in the skill

Available skills:
- wellness-assessment: For comprehensive wellness evaluations
- meal-planning: For creating personalized meal plans

If no skill matches, use your general wellness knowledge."""
)

print("Skill-aware agent created!")

Skill-aware agent created!


In [31]:
# Test with a skill-appropriate request
TODO_STORE.clear()

result = skill_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "I'd like a wellness assessment. I'm a 25-year-old office worker who sits most of the day, work 70 hours a week, and wants to lose 10 pounds. I'm vegan and have no major health conditions."
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Now let me provide you with specific, actionable recommendations based on your assessment:

## Your Wellness Assessment Summary

Based on your profile, you have several strengths to build on - you're young, healthy, and already following a plant-based diet. However, your 70-hour work week and sedentary lifestyle are the primary barriers to achieving your 10-pound weight loss goal.

## Immediate Action Items (Start Today):

1. **Hourly Movement Breaks:** Set a timer for every hour and do 2 minutes of movement (desk stretches, walk to water fountain, or bodyweight squats). This directly counteracts prolonged sitting.

2. **Optimize Your Workspace:** Raise your monitor to eye level and consider a standing desk converter or stability ball chair for part of your day.

3. **Prep Sunday Strategy:** Dedicate 2 hours on Sunday to batch-prep healthy vegan meals and snacks for the week to avoid relying on processed foods during your busy schedule.

## Short-Term Goals (1-2 weeks):

## Task 9: Using deepagents-cli

The `deepagents-cli` provides an interactive terminal interface for working with Deep Agents.

### Installation

```bash
uv pip install deepagents-cli
# or
pip install deepagents-cli
```

### Key Features

| Feature | Description |
|---------|-------------|
| **Interactive Sessions** | Chat with your agent in the terminal |
| **Conversation Resume** | Pick up where you left off |
| **Human-in-the-Loop** | Approve or reject agent actions |
| **File System Access** | Agent can read/write to your filesystem |
| **Remote Sandboxing** | Run in isolated Docker containers |

### Basic Usage

```bash
# Start an interactive session
deepagents

# Resume a previous conversation
deepagents --resume

# Use a specific model
deepagents --model openai:gpt-4o

# Enable human-in-the-loop approval
deepagents --approval-mode full
```

### Example Session

```
$ deepagents

Welcome to Deep Agents CLI!

You: Create a 7-day meal plan for a vegetarian athlete

Agent: I'll create a comprehensive meal plan for you. Let me:
1. Research vegetarian athlete nutrition needs
2. Design balanced daily menus
3. Save the plan to a file

[Agent uses tools...]

Agent: I've created your meal plan! You can find it at:
workspace/vegetarian_athlete_meal_plan.md

You: /exit
```

In [32]:
# Check if CLI is installed
import subprocess

try:
    result = subprocess.run(["deepagents", "--version"], capture_output=True, text=True)
    print(f"deepagents-cli version: {result.stdout.strip()}")
except FileNotFoundError:
    print("deepagents-cli not installed. Install with:")
    print("  uv pip install deepagents-cli")
    print("  # or")
    print("  pip install deepagents-cli")

deepagents-cli version: deepagents 0.0.17


### Try It Yourself!

After installing the CLI, try these commands in your terminal:

```bash
# Basic interactive session
deepagents

# With a specific working directory
deepagents --workdir ./workspace

# See all options
deepagents --help
```

Sample prompts to try:
1. "Create a weekly workout plan and save it to a file"
2. "Research the health benefits of meditation and summarize in a report"
3. "Analyze my current diet and suggest improvements" (then provide details)

## Task 10: Building a Complete Deep Agent System

Now let's bring together all four elements to build a comprehensive "Wellness Coach" system:

1. **Planning**: Track multi-week wellness programs
2. **Context Management**: Store session notes and progress
3. **Subagent Spawning**: Delegate to specialists (exercise, nutrition, mindfulness)
4. **Long-term Memory**: Remember user preferences and history

In [33]:
# Define specialized wellness subagents
# Subagents inherit the backend from the parent, so they use the same workspace
exercise_specialist = {
    "name": "exercise-specialist",
    "description": "Expert in exercise science, workout programming, and physical fitness. Use for exercise-related questions and plan creation.",
    "system_prompt": """You are an exercise specialist with expertise in:
- Workout programming for different fitness levels
- Exercise form and safety
- Progressive overload principles
- Recovery and injury prevention

Always consider the user's fitness level and any physical limitations.
Provide clear, actionable exercise instructions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

nutrition_specialist = {
    "name": "nutrition-specialist",
    "description": "Expert in nutrition science, meal planning, and dietary optimization. Use for food-related questions and meal plans.",
    "system_prompt": """You are a nutrition specialist with expertise in:
- Macro and micronutrient balance
- Meal planning and preparation
- Dietary restrictions and alternatives
- Nutrition timing for performance

Always respect dietary restrictions and preferences.
Focus on practical, achievable meal suggestions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

mindfulness_specialist = {
    "name": "mindfulness-specialist",
    "description": "Expert in stress management, sleep optimization, and mental wellness. Use for stress, sleep, and mental health questions.",
    "system_prompt": """You are a mindfulness and mental wellness specialist with expertise in:
- Stress reduction techniques
- Sleep hygiene and optimization
- Meditation and breathing exercises
- Work-life balance strategies

Be supportive and non-judgmental.
Provide practical techniques that can be implemented immediately.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

print("Specialist subagents defined!")

Specialist subagents defined!


In [34]:
# Create the Wellness Coach coordinator
wellness_coach = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        # Planning
        write_todos,
        update_todo,
        list_todos,
        # Long-term Memory
        get_user_profile,
        save_user_preference,
        # Skills
        load_skill,
    ],
    backend=filesystem_backend,  # All file ops go to workspace
    subagents=[exercise_specialist, nutrition_specialist, mindfulness_specialist],
    system_prompt="""You are a Personal Wellness Coach that coordinates comprehensive wellness programs.

## Your Role
- Understand each user's unique goals, constraints, and preferences
- Create personalized, multi-week wellness programs
- Coordinate between exercise, nutrition, and mindfulness specialists
- Track progress and adapt recommendations

## Workflow
1. **Initial Assessment**: Get user profile and understand their situation
2. **Planning**: Create a todo list for the program components
3. **Delegation**: Use specialists for domain-specific content:
   - exercise-specialist: Workout plans and fitness guidance
   - nutrition-specialist: Meal plans and dietary advice
   - mindfulness-specialist: Stress and sleep optimization
4. **Integration**: Combine specialist outputs into a cohesive program
5. **Documentation**: Save all plans and recommendations to files

## Important
- Always check user profile first for context
- Respect any medical conditions or dietary restrictions
- Provide clear, actionable recommendations
- Save progress to files so users can reference later"""
)

print("Wellness Coach created with all 4 Deep Agent elements!")

Wellness Coach created with all 4 Deep Agent elements!


In [ ]:
# Test the complete system
TODO_STORE.clear()

result = wellness_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi! My user_id is user_alex. I'd like you to create a 2-week wellness program for me.

I want to focus on:
1. Building a consistent exercise routine (I can exercise 3x per week for 30 mins)
2. Improving my diet (remember I'm vegetarian)
3. Better managing my work stress and improving my sleep

Please create comprehensive plans for each area and save them as separate files I can reference."""
    }]
})

print("Wellness Coach response:")
print(result["messages"][-1].content)

Wellness Coach response:
Perfect! Alex, I've created your comprehensive 2-week wellness program! Here's what I've delivered:

## 🎯 Your Complete Wellness Program

I've created **4 separate reference files** you can access anytime:

### 1. **Integrated Daily Schedule** 📅
`/Alex_2Week_Integrated_Wellness_Program.txt` - Your master plan combining all elements with day-by-day guidance

### 2. **Exercise Program** 💪  
`/exercise_programs/alex_2_week_exercise_program.txt` - 30-minute evening workouts, 3x per week with progressions

### 3. **Vegetarian Nutrition Plan** 🥗
`/data/vegetarian_nutrition_program_for_alex.txt` - Complete meal plans, grocery lists, and energy/sleep-supporting foods

### 4. **Stress & Sleep Program** 🧘
`/stress_management_sleep_optimization_program.txt` - Daily techniques for work stress and better sleep

## 🌟 Program Highlights

**Tailored for You:**
- ✅ Evening workouts (your preference)  
- ✅ Vegetarian-friendly meals
- ✅ Mild anxiety considerations
- ✅ Busy schedu

In [37]:
# Review what was created
print("=" * 60)
print("FINAL TODO STATUS")
print("=" * 60)
print(list_todos.invoke({}))

print("\n" + "=" * 60)
print("GENERATED FILES")
print("=" * 60)
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

FINAL TODO STATUS
✅ [todo_1] Create 2-week exercise routine plan (completed)
✅ [todo_3] Develop vegetarian meal plan and nutrition guidelines (completed)
✅ [todo_5] Design stress management and sleep optimization program (completed)
✅ [todo_7] Integrate all components into cohesive wellness program (completed)
✅ [todo_9] Save comprehensive program files (completed)

GENERATED FILES
  [FILE] Alex_2Week_Integrated_Wellness_Program.txt (7770 bytes)
  [FILE] alexs_2_week_vegetarian_meal_plan.txt (5245 bytes)
  [DIR] data/
  [FILE] exercise_program_alex.txt (4087 bytes)
  [DIR] exercise_programs/
  [FILE] morning_routine_guide.md (45226 bytes)
  [FILE] personalized_sleep_improvement_plan.md (5510 bytes)
  [DIR] research/
  [FILE] stress_management_sleep_optimization_plan_for_Alex.txt (3478 bytes)
  [FILE] stress_management_sleep_optimization_program.txt (6961 bytes)
  [DIR] workspace/


In [38]:
# Read one of the generated files
files = list(WORKSPACE.glob("*.md"))
if files:
    print(f"\nContents of {files[0].name}:")
    print("=" * 60)
    print(files[0].read_text()[:2000] + "..." if len(files[0].read_text()) > 2000 else files[0].read_text())


Contents of personalized_sleep_improvement_plan.md:
# Personalized Sleep Improvement Plan

## Your Current Sleep Challenges
- **Inconsistent bedtime**: 10pm-1am (3-hour variation)
- **Technology interference**: Phone use in bed
- **Poor morning energy**: Feeling tired upon waking

## Your Sleep Goals
- Establish a consistent sleep schedule
- Eliminate phone use in bed
- Wake up feeling refreshed and energized
- Develop sustainable sleep habits

---

## Phase 1: Foundation Building (Weeks 1-2)
**Goal**: Establish baseline and create optimal sleep environment

### Week 1 Actions:
- [ ] **Sleep tracking**: Record current bedtime, wake time, and energy level (1-10) daily
- [ ] **Environment setup**: 
  - Set bedroom temperature to 65-68°F
  - Install blackout curtains or get eye mask
  - Move phone charger outside bedroom
  - Get traditional alarm clock
- [ ] **Morning light**: Get 15-30 minutes of bright light within 1 hour of waking

### Week 2 Actions:
- [ ] **Consistent wake time**: C

---
## ❓ Question #3:

What are the key considerations when designing **subagent configurations**?

Consider:
- When should subagents share tools vs have distinct tools?
- How do you decide which model to use for each subagent?
- What's the right granularity for subagent specialization?

##### Answer:
- Subgents can share general purpose tools like file system tools, websearch tool etc. For any specific tasks that the subagent does that are unique to it, the tools are distinct from other subagents.  
- Depending on the task; For example, for a medical or finance domain specific task, we can use domain specific models such as medGemma, FinGPT etc. Also as the subagents typically don't have to do a lot of thinking/reasoning, can use smaller models to be more efficient.
- Right granularity is based on the distinct expertise required and the context that is required. We don't want too many subagents as it would just increase communication overload.

## ❓ Question #4:

For a **production wellness application** using Deep Agents, what would you need to add?

Consider:
- Safety guardrails for health advice
- Persistent storage (not in-memory)
- Multi-user support and isolation
- Monitoring and observability
- Cost management with subagents

##### Answer:
- For safety, 
   - we should add disclaimers for any health advice provided
   - do not store sensitive information (or put hard data controls like encryption etc.)
- isolate memory for different users
- persist storage in a secured database
- Add tracing, obervability with Langsmith
- Put gaurdrails on subagents - response limits, call limits etc, for cost management

---
## 🏗️ Activity #2: Build a Wellness Coach Agent

Build your own wellness coach that uses all 4 Deep Agent elements.

### Requirements:
1. **Planning**: Create todos for a 30-day wellness challenge
2. **Context Management**: Store daily check-in notes
3. **Subagents**: At least 2 specialized subagents
4. **Memory**: Remember user preferences across interactions

### Challenge:
Create a "30-Day Wellness Challenge" system that:
- Generates a personalized 30-day plan
- Tracks daily progress
- Adapts recommendations based on feedback
- Saves a weekly summary report

In [35]:
### YOUR CODE HERE ###

# Step 1: Define your subagent configurations
habits_specialist = {
    "name": "habits-specialist",
    "description": "Expert in habit formation and behavior change. Use for creating personalized habit plans.",
    "system_prompt": """You are a habits specialist with expertise in:
- Behavioral psychology
- Habit formation theories
- Behavioral change techniques
- Personalized habit plans

Always consider the user's goals and preferences when creating habit plans.
Provide clear, actionable recommendations.""",

    "tools": [],
    "model": "openai:gpt-4o-mini",
}

yoga_and_meditation_specialist = {
    "name": "yoga-and-meditation-specialist",
    "description": "Expert in yoga and meditation practices. Use for yoga and meditation-related questions and plans.",
    "system_prompt": """You are a yoga and meditation specialist with expertise in:
- Yoga poses and sequences
- Meditation techniques 
- Stress reduction techniques
- Sleep optimization

Be supportive and non-judgmental.
Provide practical techniques that can be implemented immediately.""",

    "tools": [],
    "model": "openai:gpt-4o-mini",
}

vegan_nutrition_specialist = {
    "name": "vegan-nutrition-specialist",
    "description": "Expert in vegan nutrition and meal planning. Use for vegan diet-related questions and plans.",
    "system_prompt": """You are a vegan nutrition specialist with expertise in:
- Vegan nutrition
- Macro and micronutrient balance
- Meal planning and preparation
- Dietary restrictions and alternatives
- Nutrition timing for performance

Always respect dietary restrictions and preferences.
Focus on practical, achievable meal suggestions.""",    

    "tools": [],
    "model": "openai:gpt-4o-mini",
}


# Step 2: Create any additional tools you need

wellness_challenge_tools = [write_todos, update_todo, list_todos, 
load_skill, 
get_user_profile, save_user_preference,
read_health_wellness_guide]

# Step 3: Build the main coordinator agent
wellness_challenge_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=wellness_challenge_tools,
    backend=filesystem_backend,
    subagents=[habits_specialist, yoga_and_meditation_specialist, vegan_nutrition_specialist],
    system_prompt="""You are a wellness challenge coordinator. Your job is to:
1. Create a todo list for the challenge components
2. Delegate tasks to the appropriate subagents
3. Integrate subagent outputs into a cohesive challenge plan
4. Save all plans and recommendations to files 

Before creating plans or delegating to subagents, read the wellness guide with 
read_health_wellness_guide and use that content to keep recommendations accurate and consistent.

For meal plans, load the skill meal-planning (call load_skill("meal-planning")). 

Use expert subagents for relevant techniques and knowledge:
-  yoga_and_meditation_specialist for yoga and meditation information, 
-  vegan_nutrition_specialist for vegan nutrition information, 
-  habits_specialist for habit formation information.

## Important
- Always check user profile first for context
- Respect any medical conditions or dietary restrictions
- Provide clear, actionable recommendations
- Save progress to files so users can reference later

## FIRST: Check-in vs creating a new challenge
**Before doing anything else, decide the user's intent.**
- **CHECK-IN** (do this only—do NOT create or regenerate the challenge): User says "this is my first day", "this is my second day", "day 1", "day 2", "day N", "daily check-in", or "checking in". That means they are already in the challenge and are reporting on a day. Do NOT call subagents to create a plan. Do NOT write a new challenge file. Only: read wellness journal (if exists), record this check-in to the journal, adapt recommendations, respond briefly.
- **CREATE CHALLENGE** (full flow): User explicitly asks to "create" a "30-day challenge" or "30-day wellness challenge" (e.g. "I'd like you to create a 30-day wellness challenge"). Then and only then: create todos, delegate to subagents, write challenge plan file.
- If in doubt (e.g. user says "first day"), treat as **check-in only**. You can verify by using ls to see if a challenge file (e.g. {user_id}_30day_wellness_challenge.md) already exists in workspace; if the user says "day N", the challenge already exists.

When the user does a daily check-in: First read their wellness journal if it exists 
(e.g. read_file with path "{user_id}_wellness_journal.md" or similar in the workspace) to see prior check-ins and patterns. 
Then record the new check-in (e.g. append or save to that same wellness journal file). 
If the journal file does not exist yet, create it by writing the first check-in (write_file creates the file). 
Use their feedback (what they did, missed, or how they feel) to adapt recommendations: 
e.g. suggest a lighter or modified workout, an extra rest day, or a small change to the plan. 
Respond with concrete, updated next steps, not just encouragement.

## Weekly summary report
When the user has completed a week (e.g. day 7, 14, 21, or 28) or asks for a weekly summary: read their wellness journal, summarize progress and patterns for that week, and save a **weekly summary report** to a file (e.g. {user_id}_weekN_summary.md in the workspace). Include: what went well, what was challenging, and suggested focus for the next week.

"""
)



In [32]:
# Step 4: Test with a user creating their 30-day challenge
user_id = "user_sarah"
profile_namespace = (user_id, "profile")

memory_store.put(profile_namespace, "name", {"value": "Sarah"})
memory_store.put(profile_namespace, "age", {"value": 60})
memory_store.put(profile_namespace, "goal", {"primary": "increase energy levels", "secondary": "improve mood"})
memory_store.put(profile_namespace, "conditions", {"medical": "I have a chronic condition that affects my energy levels"})
memory_store.put(profile_namespace, "preferences", {
    "diet": "vegan",
    "exercise_time": "morning",
    "sleep_schedule": "7:00 AM"
})

TODO_STORE.clear()

result = wellness_challenge_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi! My user_id is user_sarah. I'd like you to create a 
        30-day wellness challenge for me."""
    }]
})

print("Wellness Challenge response:")
print(result["messages"][-1].content)


Wellness Challenge response:
Perfect! I've completed your comprehensive 30-day wellness challenge, Sarah! Here's what I've created for you:

## 🌟 Your Complete 30-Day Wellness Challenge Package

### **Main Challenge Guide** (`user_sarah_30day_wellness_challenge.md`)
- **Week-by-week progression** designed for your energy levels and chronic condition
- **Daily structure** optimized for your morning preference
- **Gentle yoga sequences** with chair modifications for low-energy days
- **Meditation practices** progressing from 5 to 15 minutes
- **Troubleshooting guide** for challenging days

### **Daily Tracking Journal** (`user_sarah_wellness_journal.md`)
- **Daily check-in pages** with energy and mood tracking (1-5 scale)
- **Habit completion checklists** 
- **Meal logging sections**
- **Reflection prompts** to identify patterns
- **Weekly and monthly summary templates**

### **Personalized Meal Plan** (`user_sarah_meal_plan.md`)
- **4 weekly themes** focusing on different energy-boostin

In [ ]:
# Step 5: Simulate a daily check-in and adaptation
# Day 1 checkin
day1_checkin = wellness_challenge_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi! My user_id is user_sarah. This is my first day of the challenge. I'm feeling great today!"""
    }]
})

print("Day 1 checkin response:")
print(day1_checkin["messages"][-1].content)
print("\n\n")

# Day 2 checkin
day2_checkin = wellness_challenge_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi! My user_id is user_sarah. This is my second day of the challenge. 
        I did my morning meditation and stuck to the meal plan, 
        but I missed yesterday’s workout and I’m feeling tired."""
    }]
})

print("Day 2 checkin response:")
print(day2_checkin["messages"][-1].content)
print("\n\n")

# Day 3 checkin
day3_checkin = wellness_challenge_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi! My user_id is user_sarah. This is my third day of the challenge. 
        Feel restless to do meditation today, but I did my workout and ate well."""
    }]
})  

print("Day 3 checkin response:")
print(day3_checkin["messages"][-1].content)
print("\n\n")





Day 1 checkin response:
Wonderful, Sarah! I've created your wellness journal and recorded your first day check-in. Since you're feeling great and energetic today, you're in the perfect mindset to start your 30-day wellness challenge!

## Your Day 1 Focus (Week 1 - Gentle Foundations):

**Start with your morning routine:**
1. **Hydrate** - Drink that glass of room temperature water 
2. **Gentle stretching** - Just 5-10 minutes focusing on neck, shoulders, and back
3. **Breathing exercise** - 3 minutes of deep breathing to set a positive tone
4. **Gratitude** - Write down one thing you're grateful for (maybe that great energy you're feeling!)

**Your 10-minute yoga sequence:**
- Cat-Cow Pose (5-7 breaths each)
- Child's Pose (1-2 minutes with cushion support)  
- Seated Forward Bend (modify as comfortable)
- End with 5 minutes of simple breath awareness meditation

**Nutrition reminders:**
Your meal plan includes energizing vegan options like overnight oats for breakfast, quinoa salad fo

---
## Summary

In this session, we explored **Deep Agents** and their four key elements:

| Element | Purpose | Implementation |
|---------|---------|----------------|
| **Planning** | Track complex tasks | `write_todos`, `update_todo`, `list_todos` |
| **Context Management** | Handle large contexts | File system tools, automatic offloading |
| **Subagent Spawning** | Delegate to specialists | `task` tool with custom configs |
| **Long-term Memory** | Remember across sessions | LangGraph Store integration |

### Key Takeaways:

1. **Deep Agents handle complexity** - Unlike simple tool loops, they can manage long-horizon, multi-step tasks
2. **Planning is context engineering** - Todo lists and files aren't just organization—they're extended memory
3. **Subagents prevent context bloat** - Delegation keeps the main agent focused and efficient
4. **Skills enable progressive disclosure** - Load capabilities on-demand instead of upfront
5. **The CLI makes interaction natural** - Interactive sessions with conversation resume

### Deep Agents vs Traditional Agents

| Aspect | Traditional Agent | Deep Agent |
|--------|-------------------|------------|
| Task complexity | Simple, single-step | Complex, multi-step |
| Context management | All in conversation | Files + summaries |
| Delegation | None | Subagent spawning |
| Memory | Within thread | Across sessions |
| Planning | Implicit | Explicit (todos) |

### When to Use Deep Agents

**Use Deep Agents when:**
- Tasks require multiple steps or phases
- Context would overflow in a simple loop
- Specialization would improve quality
- Users need to resume sessions
- Long-term memory is valuable

**Use Simple Agents when:**
- Tasks are straightforward Q&A
- Single tool call suffices
- Context fits easily
- No need for persistence
### Further Reading

- [Deep Agents Documentation](https://docs.langchain.com/oss/python/deepagents/overview)
- [Deep Agents GitHub](https://github.com/langchain-ai/deepagents)
- [Context Management Blog Post](https://www.blog.langchain.com/context-management-for-deepagents/)
- [Building Multi-Agent Applications](https://www.blog.langchain.com/building-multi-agent-applications-with-deep-agents/)
- [LangGraph Memory Concepts](https://langchain-ai.github.io/langgraph/concepts/memory/)